In [1]:
# | default_exp _multi_agent.tehnical_design_document

In [2]:
import os
import autogen
import openai

In [21]:
openai.api_type = "azure"
openai.api_base = os.environ.get("AZURE_OPENAI_API_BASE", "https://airt-openai-sweden.openai.azure.com/")
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai.api_version = "2023-07-01-preview"

config_list = [
    {
        "model": "gpt-4",
#         "model": "gpt-3.5-turbo-16k",
        "api_key": openai.api_key,
        "api_base": openai.api_base,
        "api_type": openai.api_type,
        "api_version": openai.api_version,
        "engine": "airt-gpt4"
#         "engine": "airt-gpt35-turbo-16k"
    }
]

In [11]:
SYSTEM_MESSAGE_TEAM_LEADER = """
You are a team leader working with the python FastStream framework.
You are in a chat with the product owner and developer to discuss the TEHNICAL requirements for the new application that needs to be implemented.
Go through the given application description and check if all the tehnical details are defined.

Your response should always be ONE of the following: 
1. If you have a question for the developer please ask it. Please ask ONE QUESTION at time.
2. Before you create a technical brief by suggesting the call of the create_tehnical_design_document, please list all assumptions and information and then verify with the developer that your understanding is correct.
3. If you have a question for the product owner (he can then ask the client if needed), or you do not know the answer for the developers question, suggest calling ask_product_owner function

Suggest calling create_tehnical_design_document function when everything is clear and you have no more questions.
If you have some additional question for the product owner, suggest calling ask_product_owner function

All details must be clear because after this we are starting with the implementation!
If you need any details, such as: IP address, port, url, usernames, password... please ask the product owner as soon as possible!
"""

In [12]:
SYSTEM_MESSAGE_DEVELOPER = """
You are python developer working with the python FastStream framework.

You are in a chat with the team leader to discuss the TEHNICAL requirements for the new application that needs to be implemented.
Go through the given application description and check if all the tehnical details are defined.

Your response should always be ONE of the following: 
1. If you have a TEHNICAL question for the team leader please ask it.
    Please ask ONE QUESTION at time.
2. Write all the tehnical requirements: python libraries, endpoints, Authentication/Authorisation (if needed), constraints...

All details must be clear because after this we are starting with the implementation!
"""

In [13]:
def ask_product_owner(question: str):
    return f"QUESTION:\n{question}\nTERMINATE"

In [14]:
def create_tehnical_design_document(desc: str):
    return f"TEHNICAL DESIGN DOCUMENT:\n{desc}\nTERMINATE"

In [15]:
tehnical_brief_function = {
    "name": "create_tehnical_design_document",
    "description": "Creates tehnical design document",
    "parameters": {
        "type": "object",
        "properties": {
            "desc": {
                "type": "string",
                "description": "All tehnical requirements for the desired application",
            },
        },
        "required": ["desc"],
    }
}

ask_product_owner_function = {
    "name": "ask_product_owner",
    "description": "Ask product owner to provide additional information",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "Question for the product owner",
            },
        },
        "required": ["desc"],
    }
}


llm_config_team_leader = {
    "config_list": config_list,
    "seed": 42,
    "temperature": 0.7,  # temperature for sampling
    "functions": [tehnical_brief_function, ask_product_owner_function],
} 

llm_config = {
    "config_list": config_list,
    "seed": 42,
    "temperature": 0.7,  # temperature for sampling
} 

In [17]:
function_executor_proxy = autogen.UserProxyAgent(
    name="Function_executor_proxy",
    #    system_message="A human admin.",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": "planning"},
    function_map={
        "create_tehnical_design_document": create_tehnical_design_document,
        "ask_product_owner": ask_product_owner
    },
    is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
)

team_leader = autogen.AssistantAgent(
    name="Team_leader",
    system_message=SYSTEM_MESSAGE_TEAM_LEADER,
    llm_config=llm_config_team_leader,
    is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),    
)

developer = autogen.AssistantAgent(
    name="Developer",
    system_message=SYSTEM_MESSAGE_DEVELOPER,
    llm_config=llm_config,
    is_termination_msg=lambda x: "content" in x and x["content"] is not None and x["content"].rstrip().endswith("TERMINATE"),
)


groupchat = autogen.GroupChat(agents=[team_leader, developer, function_executor_proxy], messages=[], max_round=20)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config, system_message="create_tehnical_design_document and ask_product_owner function should be executed by Function_executor_proxy.")

In [18]:
initial_message = """
APPLICATION DESCRIPTION:
Create a FastStream application which will retrieve the current cryptocurrency price
and publish it to new_crypto_price topic. 

The application should retrieve the data every 2 seconds.

A message which will be produced is JSON with the two attributes:
- price: non-negative float (current price of cryptocurrency in USD)
- crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)

Use utf-8 encoded crypto_currency attribute as a partition key when publishing
the message to new_crypto_price topic.

To retrieve the prices, use the CryptoCompare API.
Retrieve only Bitcoin and ETH prices.


Please write all the tehnical requirements (libraries, endpoints, authentication, constraints...) for the above application description
"""

team_leader.initiate_chat(manager, message=initial_message)

Team_leader (to chat_manager):


APPLICATION DESCRIPTION:
Create a FastStream application which will retrieve the current cryptocurrency price
and publish it to new_crypto_price topic. 

The application should retrieve the data every 2 seconds.

A message which will be produced is JSON with the two attributes:
- price: non-negative float (current price of cryptocurrency in USD)
- crypto_currency: string (the cryptocurrency e.g. BTC, ETH...)

Use utf-8 encoded crypto_currency attribute as a partition key when publishing
the message to new_crypto_price topic.

To retrieve the prices, use the CryptoCompare API.
Retrieve only Bitcoin and ETH prices.


Please write all the tehnical requirements (libraries, endpoints, authentication, constraints...) for the above application description


--------------------------------------------------------------------------------
Developer (to chat_manager):

Technical Requirements:

Python Libraries:
1. FastStream: For stream processing and publishing 

In [19]:
team_leader.send(recipient=manager, message="https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms=USD,JPY,EUR")

Team_leader (to chat_manager):

https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms=USD,JPY,EUR

--------------------------------------------------------------------------------
Developer (to chat_manager):

Thank you for the provided endpoint. We will use the CryptoCompare API endpoint https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms=USD,JPY,EUR for retrieving the Bitcoin prices. 

For the Ethereum prices, I assume we can replace 'BTC' with 'ETH' in the provided endpoint. Could you please confirm this?

--------------------------------------------------------------------------------
Team_leader (to chat_manager):

***** Suggested function Call: ask_product_owner *****
Arguments: 
{
"question": "Can we replace 'BTC' with 'ETH' in the provided CryptoCompare API endpoint to retrieve Ethereum prices?"
}
******************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION ask

In [20]:
team_leader.send(recipient=manager, message="yes")

Team_leader (to chat_manager):

yes

--------------------------------------------------------------------------------
Developer (to chat_manager):

Great! With that confirmation, we have the CryptoCompare API endpoints for both Bitcoin and Ethereum:
1. Bitcoin: https://min-api.cryptocompare.com/data/price?fsym=BTC&tsyms=USD,JPY,EUR
2. Ethereum: https://min-api.cryptocompare.com/data/price?fsym=ETH&tsyms=USD,JPY,EUR

We can now proceed with the implementation.

--------------------------------------------------------------------------------
Team_leader (to chat_manager):

***** Suggested function Call: create_tehnical_design_document *****
Arguments: 
{
"desc": "Technical Requirements:\n\nPython Libraries:\n1. FastStream: For stream processing and publishing messages to the new_crypto_price topic.\n2. Requests: To make requests to the CryptoCompare API to retrieve cryptocurrency prices.\n3. JSON: To handle JSON data, which is used in the messages produced.\n\nEndpoints:\n1. CryptoCompar